In [ ]:
import polars as pl


nombre_archivo = "competencia_01_crudo.csv"
df = pl.read_csv(
    nombre_archivo,
    infer_schema_length=100000  # mirar más filas para inferir (con 10000 fallaba)
)

print(df['foto_mes'].value_counts())

# 2. Función para agregar solo clase_ternaria
def agregar_clase_ternaria_optimizada(df: pl.DataFrame) -> pl.DataFrame:
    """
    Versión optimizada usando Polars.
    Solo devuelve clase_ternaria sin dejar columnas intermedias.
    """
    max_foto_mes = df["foto_mes"].max()

    return df.lazy().sort(["numero_de_cliente", "foto_mes"]).with_columns([
        pl.when(
            (pl.col("foto_mes").shift(-1).over("numero_de_cliente").is_null()) & 
            (pl.col("foto_mes") != max_foto_mes)
        ).then(pl.lit("BAJA+1"))
        .when(
            (pl.col("foto_mes").shift(-2).over("numero_de_cliente").is_null()) & 
            (pl.col("foto_mes") <= max_foto_mes - 2)
        ).then(pl.lit("BAJA+2"))
        .otherwise(pl.lit("CONTINUA"))
        .alias("clase_ternaria")
    ]).collect()

# 3. Agregar la columna clase_ternaria al dataframe
df = agregar_clase_ternaria_optimizada(df)

# 4. Tabla de frecuencia de clase_ternaria por foto_mes
tabla = (
    df.pivot(
        values="numero_de_cliente",
        index="foto_mes",
        on="clase_ternaria",
        aggregate_function="len"
    )
    .fill_null(0)
)

print(tabla)


df.write_csv(nombre_archivo)

AssertionError: 

AssertionError: 